In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, SimpleRNN, Dense, TimeDistributed
from tensorflow.keras.models import Model

import wandb
from wandb.keras import WandbCallback

warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")
warnings.filterwarnings("ignore", category=UserWarning, module="absl")

In [3]:
tf.config.list_physical_devices('GPU'), tf.test.gpu_device_name()

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')],
 '/device:GPU:0')

In [4]:
# Loading the DATA
train_data = pd.read_csv("dakshina_dataset_v1.0/kn/lexicons/kn.translit.sampled.train.tsv", sep='\t', header=None)
test_data =  pd.read_csv("dakshina_dataset_v1.0/kn/lexicons/kn.translit.sampled.test.tsv", sep='\t', header = None)
val_data = pd.read_csv("dakshina_dataset_v1.0/kn/lexicons/kn.translit.sampled.dev.tsv", sep='\t', header= None)

In [5]:
# Filtering the data
train_data = train_data.dropna()
test_data = test_data.dropna()
val_data = val_data.dropna()

In [6]:
# Creating Vocubalory
X = [list('\t'+ w +'\n') for w in np.array(train_data[1])] # Input Sequence
Y = [list('\t'+ w +'\n') for w in np.array(train_data[0])] # Traget Sequence

kannada_vocab = set() # Set avoids duplicates
eng_vocab = set() # Set avoids duplicates

for word in X:
    for char in word:
        eng_vocab.add(char)

for word in Y:
    for char in word:
        kannada_vocab.add(char)

kannada_vocab_list = sorted(list(kannada_vocab))
eng_vocab_list = sorted(list(eng_vocab))

In [7]:
# Preparing Data
num_input_tokens = len(eng_vocab_list)
num_output_tokens = len(kannada_vocab_list)

# Mapping character to index
input_token_index = {}
for i, char in enumerate(eng_vocab_list):
    input_token_index[char] = i

output_token_index = {}
for i, char in enumerate(kannada_vocab_list):
    output_token_index[char] = i

# Mapping index to character: 
# Inverting the previous dictionaries to avoid searching operations while decoding

input_index_token = {i: char for char, i in input_token_index.items()}
output_input_token = {i: char for char, i in output_token_index.items()}

max_input_seq_length = max([len(seq) for seq in X])
max_output_seq_length = max([len(seq) for seq in Y])

In [8]:
def prepare(x, y):
    # Converts list of characters -> list of indexes and provides the corresponding teacher forcing targets
    decoder_target = np.zeros((len(x), max_output_seq_length, num_output_tokens), dtype="float32")

    for i, seq in enumerate(y):
        for t, char in enumerate(seq):
            if t > 0 :
                decoder_target[i, t-1, output_token_index[char]] = 1.0
        decoder_target[i, t:, output_token_index["\n"]] = 1.0
    
    encoder_samples = [[input_token_index[letter] for letter in seq] for seq in x]
    decoder_samples = [[output_token_index[letter] for letter in seq] for seq in y]

    return encoder_samples, decoder_samples, decoder_target

In [9]:
X_train = [list('\t'+ w +'\n') for w in np.array(train_data[1])] # Input Sequence
Y_train = [list('\t'+ w +'\n') for w in np.array(train_data[0])] # Traget Sequence

X_test = [list('\t'+ w +'\n') for w in np.array(test_data[1])] # Input Sequence
Y_test = [list('\t'+ w +'\n') for w in np.array(test_data[0])] # Traget Sequence

X_val = [list('\t'+ w +'\n') for w in np.array(val_data[1])] # Input Sequence
Y_val = [list('\t'+ w +'\n') for w in np.array(val_data[0])] # Traget Sequence

In [10]:
encoder_train, decoder_train, decoder_target_train = prepare(X_train, Y_train)
encoder_test, decoder_test, decoder_target_test = prepare(X_test, Y_test)
encoder_val, decoder_val, decoder_target_val = prepare(X_val, Y_val)

In [11]:
def pad(List, max_length):
    return sequence.data_utils.pad_sequences(List, maxlen=max_length, padding="post")

In [12]:
encoder_train = pad(encoder_train, max_input_seq_length)
encoder_test = pad(encoder_test, max_input_seq_length)
encoder_val = pad(encoder_val, max_input_seq_length)

decoder_train = pad(decoder_train, max_output_seq_length)
decoder_test = pad(decoder_test, max_output_seq_length)
decoder_val = pad(decoder_val, max_output_seq_length)

In [13]:
def Eng2Kan(encoder_tokens, decoder_tokens, embedding_dim, hidden_dim, max_input_seq_length, max_output_seq_length,
            no_encoder_layers, no_decoder_layers, dropouts=0.1, cell_type="gru"):

    cell_type = cell_type.lower()  
    
    input_encoder = Input(shape=(None,))
    input_decoder = Input(shape=(None,))
    
    input_embedding = Embedding(encoder_tokens, embedding_dim, input_length=max_input_seq_length)(input_encoder)
    x = input_embedding

    encoder_states = None
    for _ in range(no_encoder_layers):
        if cell_type == "lstm":
            x, memory_state, carry_state = LSTM(hidden_dim, return_state=True, return_sequences=True, 
                                                dropout=dropouts)(x)
            encoder_states = [memory_state, carry_state]
        elif cell_type == "gru":
            x, carry_state = GRU(hidden_dim, return_state=True, return_sequences=True, dropout=dropouts)(x)
            encoder_states = [carry_state]
        else:
            x, carry_state = SimpleRNN(hidden_dim, return_state=True, return_sequences=True, dropout=dropouts)(x)
            encoder_states = [carry_state]

    target_embedding = Embedding(decoder_tokens, embedding_dim, input_length=max_output_seq_length)(input_decoder)

    if cell_type == "lstm":
        decoder_x, _, _ = LSTM(hidden_dim, return_sequences=True, return_state=True,
                               dropout=dropouts)(target_embedding, initial_state=encoder_states)
    elif cell_type == "rnn":
        decoder_x, _ = SimpleRNN(hidden_dim, return_sequences=True, return_state=True,
                                 dropout=dropouts)(target_embedding, initial_state=encoder_states)
    elif cell_type == "gru":
        decoder_x, _ = GRU(hidden_dim, return_sequences=True, return_state=True,
                           dropout=dropouts)(target_embedding, initial_state=encoder_states)

    for _ in range(no_decoder_layers - 1):
        if cell_type == "lstm":
            decoder_x, _, _ = LSTM(hidden_dim, return_sequences=True, return_state=True,
                                   dropout=dropouts)(decoder_x)
        elif cell_type == "rnn":
            decoder_x, _ = SimpleRNN(hidden_dim, return_sequences=True, return_state=True,
                                     dropout=dropouts)(decoder_x)
        elif cell_type == "gru":
            decoder_x, _ = GRU(hidden_dim, return_sequences=True, return_state=True,
                               dropout=dropouts)(decoder_x)

    output = TimeDistributed(Dense(decoder_tokens, activation='softmax'))(decoder_x)

    model = Model([input_encoder, input_decoder], output)

    return model

In [14]:
class Word_Accuracy(tf.keras.callbacks.Callback):
    def __init__(self, encoder_cv, decoder_cv, inv_target_token_index, max_length_y, sample_size=None):
        super().__init__()
        self.encoder_val = encoder_cv
        self.decoder_val = decoder_cv
        self.inv_target_token_index = inv_target_token_index
        self.max_output_seq_length = max_length_y
        self.sample_size = sample_size  # optional

    def on_epoch_end(self, epoch, logs=None):
        if self.sample_size:
            indices = np.random.choice(len(self.encoder_val), self.sample_size, replace=False)
            enc = self.encoder_val[indices]
            dec = self.decoder_val[indices]
        else:
            enc = self.encoder_val
            dec = self.decoder_val

        predictions = self.model.predict([enc, dec], verbose=0)
        count = 0

        for i in range(len(predictions)):
            actual_out = re.sub(r'\t|\n', '', ''.join(
                [self.inv_target_token_index[dec[i][j]] for j in range(self.max_output_seq_length)]
            ))
            predicted_out = re.sub(r'\t|\n', '', ''.join(
                [self.inv_target_token_index[np.argmax(predictions[i][j])] for j in range(self.max_output_seq_length)]
            ))

            if actual_out == predicted_out:
                count += 1

        acc = count / len(predictions)
        logs["WordAccuracy"] = acc
        print(f"\nEpoch {epoch + 1}: Word-level Accuracy = {acc:.4f}")

        wandb.log({"Val_WordAccuracy": acc, "epoch": epoch + 1})


In [15]:
sweep_config = {
    "name": "RNN - Hyperparameter Search",
    "metric": {
        "name": "Val_WordAccuracy", 
        "goal": "maximize"
    },
    "method": "random", # options: [bayes, random, grid]
    "parameters" : {
        "char_embedding_dim": {
            "values": [8, 12, 16, 32]
        },
        "encoder_layers" : {
            "values" : [1, 2, 3]
        },
        "decoder_layers": {
            "values": [1, 2, 3]
        },
        "hidden_state_dim":{
            "values": [64, 128, 256, 512]
        },
        "cell_type":{
            "values":["gru", "lstm", "rnn"]
        },
        "dropout": {
            "values":[0.1, 0.2, 0.3, 0.4]
        }
    }
}

In [16]:
def sweep_hyperparameters():
    
    default_config = {
        "char_embedding_dim": 32,
        "encoder_layers": 2,
        "decoder_layers": 2,
        "hidden_state_dim": 512,
        "cell_type": "lstm",
        "dropout": 0.1  
    }

    wandb.init(config = default_config)
    config = wandb.config # To get the Hyper Parameters from sweep_config

    embedding_dim = config.char_embedding_dim
    encoder_layers = config.encoder_layers
    decoder_layers = config.decoder_layers
    hidden_dim = config.hidden_state_dim
    cell_type = config.cell_type
    dropout = config.dropout

    run_name = f"RNN Layer Type: {cell_type}, Character_Embedding_dim: {embedding_dim}, Hidden_dim:{hidden_dim}, Num_Encoder_Layer:{encoder_layers}, Num_Decoder_Layer:{decoder_layers}, Dropout:{dropout}"
    print("Sweep Name: ", run_name)

    model = Eng2Kan(num_input_tokens, num_output_tokens, embedding_dim, hidden_dim, max_input_seq_length, 
                    max_output_seq_length, encoder_layers, decoder_layers, dropout, cell_type)

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

    epochs = 10
    batch_size = 32
    
    word_accuracy = Word_Accuracy(
        encoder_cv=encoder_val,
        decoder_cv=decoder_val,
        inv_target_token_index=output_input_token,
        max_length_y=max_output_seq_length
    )

    history = model.fit(
        [encoder_train, decoder_train],
        decoder_target_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=([encoder_val, decoder_val], decoder_target_val),
        callbacks=[word_accuracy, WandbCallback()],
        verbose=1
    )

    wandb.run.name = run_name
    wandb.run.save()

In [ ]:
sweep_ID = wandb.sweep(sweep_config, entity="manoj_da24s018-iit-madras", project="Transliterator_HyperParameterSweep")
wandb.agent(sweep_ID, sweep_hyperparameters, count=25)

Create sweep with ID: fjtuyyfs
Sweep URL: https://wandb.ai/manoj_da24s018-iit-madras/Transliterator_HyperParameterSweep/sweeps/fjtuyyfs


wandb: Agent Starting Run: 3t7mhwgo with config:
wandb: 	cell_type: gru
wandb: 	char_embedding_dim: 12
wandb: 	decoder_layers: 2
wandb: 	dropout: 0.3
wandb: 	encoder_layers: 2
wandb: 	hidden_state_dim: 256
wandb: Currently logged in as: manoj_da24s018 (manoj_da24s018-iit-madras). Use `wandb login --relogin` to force relogin


Sweep Name:  RNN Layer Type: gru, Character_Embedding_dim: 12, Hidden_dim:256, Num_Encoder_Layer:2, Num_Decoder_Layer:2, Dropout:0.3


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
1580/1582 [============================>.] - ETA: 0s - loss: 0.9503 - accuracy: 0.7406
Epoch 1: Word-level Accuracy = 0.0002


...

INFO:tensorflow:Assets written to: D:\DL\wandb\run-20250515_213753-tu1s1zno\files\model-best\assets
wandb: Adding directory to artifact (D:\DL\wandb\run-20250515_213753-tu1s1zno\files\model-best)... Done. 0.1s


1582/1582 [==============================] - 44s 28ms/step - loss: 0.0705 - accuracy: 0.9773 - val_loss: 0.1560 - val_accuracy: 0.9577 - WordAccuracy: 0.3853
Epoch 10/10
1580/1582 [============================>.] - ETA: 0s - loss: 0.0660 - accuracy: 0.9787
Epoch 10: Word-level Accuracy = 0.3891
1582/1582 [==============================] - 33s 21ms/step - loss: 0.0660 - accuracy: 0.9787 - val_loss: 0.1571 - val_accuracy: 0.9582 - WordAccuracy: 0.3891


Val_WordAccuracy,▁▁▄▅▆▇▇███
WordAccuracy,▁▁▄▅▆▇▇███
accuracy,▁▄▆▇▇█████
epoch,▂▁▂▂▃▂▄▃▅▄▅▅▆▅▇▆▇▇█▇
loss,█▅▃▂▁▁▁▁▁▁
val_accuracy,▁▄▆▇▇█████
val_loss,█▄▂▂▁▁▁▁▁▁
Val_WordAccuracy,0.3891
WordAccuracy,0.3891
accuracy,0.97872
best_epoch,8


wandb: Agent Starting Run: agqmc8rs with config:
wandb: 	cell_type: gru
wandb: 	char_embedding_dim: 8
wandb: 	decoder_layers: 3
wandb: 	dropout: 0.4
wandb: 	encoder_layers: 1
wandb: 	hidden_state_dim: 128


Sweep Name:  RNN Layer Type: gru, Character_Embedding_dim: 8, Hidden_dim:128, Num_Encoder_Layer:1, Num_Decoder_Layer:3, Dropout:0.4
Epoch 1/10
   4/1582 [..............................] - ETA: 40s - loss: 4.0906 - accuracy: 0.5009    